# Custom Motif Prediction Pipeline
This notebook demonstrates how to
1.  Scan the proteome for a custom regex pattern (simulating a motif).
2.  Annotate instances with AlphaMissense features.
3.  Predict pathogenicity/relevance using a trained Decision Tree

In [8]:
import sys
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Add src to path
sys.path.append(os.path.abspath('../../'))

from src.motif_prediction_pipeline.motif import scan_proteome_for_motifs
from src.motif_prediction_pipeline.features import add_alphamissense_features
from src.motif_prediction_pipeline.model import MotifPredictor
from src.config import RAW_DATA_DIR, PROCESSED_DATA_DIR, OUTPUT_DIR

## 1. Define Motif and Scan Proteome
We define a regular expression.

Example: **SH3 domain binding motif** (Class I) often follows `[RKY]..P..P`.
Or a simpler one: `P..P` (proline-rich region core).

Let's try a pattern with Key Residues defined by brackets to test our complex parser.

In [9]:
# TEST PATTERN: LIG_PCNA_PIPBox_1
REGEX_PATTERN = r"R..[ILVMF][ILMVF][^P][^P][ILVM].{4,7}L(([KR].)|(NK))[VATI]"

print(f"Scanning proteome for pattern: {REGEX_PATTERN}")

# This loads the proteome from data/processed/main_isoforms.tsv automatically
motifs_df = scan_proteome_for_motifs(regex_pattern=REGEX_PATTERN)

# IMPORTANT: Add the Regex column so features.py knows how to find Key Residues!
motifs_df['Regex'] = REGEX_PATTERN

print(f"Found {len(motifs_df)} matches.")
display(motifs_df.head())

Scanning proteome for pattern: R..[ILVMF][ILMVF][^P][^P][ILVM].{4,7}L(([KR].)|(NK))[VATI]
Loading proteome from /dlab/home/norbi/paper_projects/pathogenic-idr-analysis/data/processed/main_isoforms.tsv...
Scanning 19836 sequences for pattern: R..[ILVMF][ILMVF][^P][^P][ILVM].{4,7}L(([KR].)|(NK))[VATI]


Scanning: 100%|██████████| 19836/19836 [00:00<00:00, 21562.68it/s]

Found 90 matches.


,Protein_ID,Start,End,Matched_Sequence,ELMIdentifier,Regex
0,ACER1-201,122,140,RLVFITTVVSTLLSFLRPT,Custom_Regex,"R..[ILVMF][ILMVF][^P][^P][ILVM].{4,7}L(([KR].)..."
1,APOB-201,2085,2102,RQTIIVVLENVQRNLKHI,Custom_Regex,"R..[ILVMF][ILMVF][^P][^P][ILVM].{4,7}L(([KR].)..."
2,ARAP2-201,800,815,RKTLLASLTKEELNKA,Custom_Regex,"R..[ILVMF][ILMVF][^P][^P][ILVM].{4,7}L(([KR].)..."
3,ARF1-201,104,119,REELMRMLAEDELRDA,Custom_Regex,"R..[ILVMF][ILMVF][^P][^P][ILVM].{4,7}L(([KR].)..."
4,ARF3-201,104,119,REELMRMLAEDELRDA,Custom_Regex,"R..[ILVMF][ILMVF][^P][^P][ILVM].{4,7}L(([KR].)..."


## 2. Feature Engineering (AlphaMissense)

This step loads `am_order.tsv` and `am_disorder.tsv`, creates a lookup, and calculates scores.
It uses `regex_utils` to distinguish Key Residues (e.g., the `[RK]` and `[ST]`) from Flanking ones (the `.{2,3}`).

In [10]:
# Check if AM data exists before running (to avoid long waits if files missing)
am_ord_path = os.path.join(PROCESSED_DATA_DIR, "am_order.tsv")
am_dis_path = os.path.join(PROCESSED_DATA_DIR, "am_disorder.tsv")

if os.path.exists(am_ord_path) or os.path.exists(am_dis_path):
    print("AlphaMissense data found. calculating features...")

    # Calculate Features
    # This might take 1-2 minutes depending on the number of matches and AM data size
    features_df = add_alphamissense_features(motifs_df)

    print("\nFeature extraction complete.")
    display(features_df.head())
else:
    print("CRITICAL WARNING: AlphaMissense processed files not found in data/processed/")
    print("Please run setup_data.py or download the data first.")
    # Stop execution or generate dummy data for testing logic
    features_df = motifs_df.copy()
    cols = MotifPredictor().feature_cols
    for c in cols: features_df[c] = np.random.rand(len(features_df))

AlphaMissense data found. calculating features...
Loading AM Order data from /dlab/home/norbi/paper_projects/pathogenic-idr-analysis/data/processed/am_order.tsv...
Loading AM Disorder data from /dlab/home/norbi/paper_projects/pathogenic-idr-analysis/data/processed/am_disorder.tsv...
Combining AM datasets...
Indexing AlphaMissense data...


Indexing: 100%|██████████| 19516/19516 [00:04<00:00, 4852.00it/s]


Calculating motif features...


Calculating: 100%|██████████| 90/90 [00:00<00:00, 2731.71it/s]



Feature extraction complete.


,Protein_ID,Start,End,Matched_Sequence,ELMIdentifier,Regex,motif_am_mean_score,AM_Max,key_residue_am_mean_score,flanking_residue_am_mean_score,sequential_am_score,Key_vs_NonKey_Difference,Motif_vs_Sequential_Difference
0,ACER1-201,122,140,RLVFITTVVSTLLSFLRPT,Custom_Regex,"R..[ILVMF][ILMVF][^P][^P][ILVM].{4,7}L(([KR].)...",0.447679,0.905016,0.286182,0.527187,0.540965,-0.241005,-0.093286
1,APOB-201,2085,2102,RQTIIVVLENVQRNLKHI,Custom_Regex,"R..[ILVMF][ILMVF][^P][^P][ILVM].{4,7}L(([KR].)...",0.294865,0.482405,0.288795,0.289581,0.370626,-0.000786,-0.075760
2,ARAP2-201,800,815,RKTLLASLTKEELNKA,Custom_Regex,"R..[ILVMF][ILMVF][^P][^P][ILVM].{4,7}L(([KR].)...",0.494145,0.874432,0.667612,0.746789,0.491825,-0.079177,0.002320
3,ARF1-201,104,119,REELMRMLAEDELRDA,Custom_Regex,"R..[ILVMF][ILMVF][^P][^P][ILVM].{4,7}L(([KR].)...",0.827500,0.989153,0.863922,0.838943,0.861677,0.024979,-0.034177
4,ARF3-201,104,119,REELMRMLAEDELRDA,Custom_Regex,"R..[ILVMF][ILMVF][^P][^P][ILVM].{4,7}L(([KR].)...",0.828118,0.987779,0.861221,0.828577,0.847112,0.032644,-0.018994


### Verify Key Residue Logic

Let's check if `key_residue_am_mean_score` is different from `motif_am_mean_score`.
If they are identical, our regex parsing might be falling back to default.

In [11]:
comparison = features_df[['Matched_Sequence', 'motif_am_mean_score', 'key_residue_am_mean_score', 'flanking_residue_am_mean_score']].head(10)
comparison['Diff_Key_Motif'] = comparison['key_residue_am_mean_score'] - comparison['motif_am_mean_score']
display(comparison)

,Matched_Sequence,motif_am_mean_score,key_residue_am_mean_score,flanking_residue_am_mean_score,Diff_Key_Motif
0,RLVFITTVVSTLLSFLRPT,0.447679,0.286182,0.527187,-0.161497
1,RQTIIVVLENVQRNLKHI,0.294865,0.288795,0.289581,-0.006070
2,RKTLLASLTKEELNKA,0.494145,0.667612,0.746789,0.173467
3,REELMRMLAEDELRDA,0.827500,0.863922,0.838943,0.036422
4,REELMRMLAEDELRDA,0.828118,0.861221,0.828577,0.033103
5,RGPVLESIVARTLRQV,0.864919,0.859033,0.768019,-0.005887
6,RKNLLELIEYTHMLRVT,0.627816,0.540676,0.590769,-0.087140
7,RVLLLGDVATLPLRKV,0.342463,0.393701,0.630749,0.051238
8,RNCLIELVNCQMVLRGA,0.682230,0.545329,0.649157,-0.136900
9,RKALVAQVIKEKLRLKSA,0.735850,0.604156,0.525259,-0.131695


##  3. Prediction with Decision Tree

We load the trained model. If no model exists yet (because we haven't run the training notebook), we train a dummy one on the fly just to demonstrate the pipeline.


In [ ]:
model_path = os.path.join(PROCESSED_DATA_DIR, "models", "decision_tree_v1.pkl")
predictor = MotifPredictor(model_filename="decision_tree_v1.pkl")

# Check if model exists
if predictor.model is None:
    print("No pre-trained model found. Training a DEMO model on current data...")
    print("(Note: In a real scenario, train the model on ClinVar Pathogenic/Benign datasets first!)")

    # Create a dummy target
    X = features_df[predictor.feature_cols].fillna(0)
    y_dummy = ((X['key_residue_am_mean_score'] > 0.6) &
               (X['Key_vs_NonKey_Difference'] > 0.1)).astype(int)

    if len(np.unique(y_dummy)) < 2:
        y_dummy = np.random.randint(0, 2, size=len(X))

    predictor.train(X, y_dummy)
else:
    print("Loaded pre-trained model.")

# Run Prediction
results_df = predictor.predict(features_df)
display(results_df.head())

## 4. Analysis & Visualization

In [6]:
# Filter High Confidence hits (using new column name)
high_conf_hits = results_df[results_df['is_PEM'] == True]

print(f"Total Scanned: {len(results_df)}")
print(f"Identified PEMs: {len(high_conf_hits)}")
print(f"Ratio: {len(high_conf_hits)/len(results_df)*100:.2f}%")

Total Scanned: 90
Identified PEMs: 41
Ratio: 45.56%


## 5. Save Results

In [7]:
output_dir = os.path.join(OUTPUT_DIR,"motif_prediction")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_file = os.path.join(output_dir, "custom_motif_prediction_results.tsv")
results_df.to_csv(output_file, sep='\t', index=False)
print(f"Results saved to: {output_file}")

Results saved to: /dlab/home/norbi/paper_projects/pathogenic-idr-analysis/output/motif_prediction/custom_motif_prediction_results.tsv
